In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Welcome\\Desktop\\project\\datascienceProject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Welcome\\Desktop\\project\\datascienceProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
import sys
import os

# Ensure we're in the project root directory
project_root = os.path.abspath('.')
if not os.path.exists(os.path.join(project_root, 'src')):
    project_root = os.path.abspath('..')
    os.chdir(project_root)

sys.path.append(project_root)  # Add project root to Python path

from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml,create_directories


In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [7]:
import os
from urllib import request
from src.datascience_project import logger
import zipfile

In [8]:
## component- Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: {headers}")

        else:
            logger.info(f"{self.config.local_data_file} already exists")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [9]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-26 10:20:51,928]: INFO: common: yaml file: config\config.yaml loaded successfully:
[2025-06-26 10:20:51,928]: INFO: common: yaml file: params.yaml loaded successfully:
[2025-06-26 10:20:51,928]: INFO: common: yaml file: schema.yaml loaded successfully:
[2025-06-26 10:20:51,943]: INFO: common: created directory at: artifacts:
[2025-06-26 10:20:51,944]: INFO: common: created directory at: artifacts/data_ingestion:
[2025-06-26 10:20:52,916]: INFO: 2652620222: artifacts/data_ingestion/data.zip download! with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 49CD:39252D:41788:F9F1C:685CD1AE
Accept-Ranges: bytes
Date: Thu,